# Imports

In [21]:
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import pandas as pd
import duckdb

In [19]:
#print(sys.path)

____

___

# Read From duckbd

In [ ]:
con = duckdb.connect("data/candles.duckdb")
#print(con.execute("SELECT * FROM candles ORDER BY timestamp DESC LIMIT 5").fetchall())
query = """SELECT * 
           FROM candles
        """
query_results = con.execute(query).fetchall()

In [14]:
print(query_results)

[(datetime.datetime(2025, 5, 12, 10, 36, 55, 214398), 1747040400.0, 104460.0, 104507.4, 104117.0, 104316.2531), (datetime.datetime(2025, 5, 12, 10, 37, 14, 475778), 1747040400.0, 104460.0, 104507.4, 104117.0, 104316.25505), (datetime.datetime(2025, 5, 12, 10, 37, 16, 6897), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.21024), (datetime.datetime(2025, 5, 12, 10, 37, 19, 56725), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.893), (datetime.datetime(2025, 5, 12, 10, 37, 28, 111583), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89343), (datetime.datetime(2025, 5, 12, 10, 37, 32, 821989), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89613), (datetime.datetime(2025, 5, 12, 10, 37, 42, 10585), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89641), (datetime.datetime(2025, 5, 12, 10, 37, 51, 671914), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.8979), (datetime.datetime(2025, 5, 12, 10, 37, 54, 956892), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89804),

In [1]:
con.close()

NameError: name 'con' is not defined

___

# Pipe the results to a Pandas Dataframe

In [15]:
df = pd.DataFrame(query_results)

In [17]:
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']

In [20]:
df.head()

,Datetime,Open,High,Low,Close,Volume
0,2025-05-12 10:36:55.214398,1.747040e+09,104460.0,104507.4,104117.0,104316.25310
1,2025-05-12 10:37:14.475778,1.747040e+09,104460.0,104507.4,104117.0,104316.25505
2,2025-05-12 10:37:16.006897,1.747040e+09,104460.0,104507.4,104117.0,104317.21024
3,2025-05-12 10:37:19.056725,1.747040e+09,104460.0,104507.4,104117.0,104317.89300
4,2025-05-12 10:37:28.111583,1.747040e+09,104460.0,104507.4,104117.0,104317.89343


____


____